# WCS log-energy transformation

This notebook is quick proove of concept, that WCS can handle equally spaced log energy axes. The specification is described in the following paper (section 3.2):

http://www.aanda.org/articles/aa/pdf/2006/05/aa3818-05.pdf


The logarithmic transform is represented by eq. (5) in the paper:

$$ S = S_r e^{w/S_r}$$

where $S_r$ corresponds to the reference value `CRVAL3`.

In [2]:
import numpy as np
import astropy.units as u
from astropy.wcs import WCS
from astropy.io import fits
from gammapy.utils.energy import Energy
from numpy.testing.utils import assert_equal

In [2]:
emin = 0.1
emax = 100
nbins = 10

# CRVAL must be set in Joule, when CUNIT3 = 'TeV' is used later, because Joule is the default
# energy unit in fits WCS
crval = (0.1 * u.J).to('TeV').value

energies = Energy.equal_log_spacing(emin, emax, nbins, unit="TeV")

def bin_width(emin, emax, nbins):
    return np.log10(emax / emin) / (nbins - 1)

In [3]:
# Define an example WCS reference header
refheader = fits.Header()
refheader['WCSAXES'] = 3
refheader['NAXIS'] = 3
refheader['CRPIX1'] = 100.5
refheader['CRPIX2'] = 100.5
refheader['CRPIX3'] = 0.5    # reference pixel for energy, 0.5 for pixel center
refheader['CDELT1'] = -0.02
refheader['CDELT2'] = 0.02  
refheader['CDELT3'] = np.log(10) * crval * bin_width(emin, emax, nbins) # log bin width
refheader['CTYPE1'] = 'RA---CAR'
refheader['CTYPE2'] = 'DEC--CAR'
refheader['CTYPE3'] = 'ENER-LOG'  # specifies the log energy axes
refheader['CUNIT1'] = 'deg'
refheader['CUNIT2'] = 'deg'
refheader['CUNIT3'] = 'TeV'
refheader['CRVAL1'] = 0
refheader['CRVAL2'] = 0
refheader['CRVAL3'] = crval        # reference value for energy

In [4]:
wcs = WCS(refheader)
_, _, indices = wcs.wcs_world2pix(0, 0, energies.value, 1)

In [5]:
assert_equal(energies.value, energies.value[indices.astype(int)])

In [3]:
# Define an example WCS reference header
refheader_tab = fits.Header()
refheader_tab['WCSAXES'] = 3
refheader_tab['NAXIS'] = 3
refheader_tab['CRPIX1'] = 100.5
refheader_tab['CRPIX2'] = 100.5
#refheader_tab['CRPIX3'] = 0.5    # reference pixel for energy, 0.5 for pixel center
refheader_tab['CDELT1'] = -0.02
refheader_tab['CDELT2'] = 0.02  
#refheader_tab['CDELT3'] = np.log(10) * crval * bin_width(emin, emax, nbins) # log bin width
refheader_tab['CTYPE1'] = 'RA---CAR'
refheader_tab['CTYPE2'] = 'DEC--CAR'
refheader_tab['CTYPE3'] = 'ENER-TAB'  # specifies the log energy axes
refheader_tab['PS3_0'] = 'ENERGIES'  # specifies the log energy axes
refheader_tab['PS3_1'] = 'ENERGY'
refheader_tab['CUNIT1'] = 'deg'
refheader_tab['CUNIT2'] = 'deg'
refheader_tab['CUNIT3'] = 'TeV'
refheader_tab['CRVAL1'] = 0
refheader_tab['CRVAL2'] = 0
#refheader_tab['CRVAL3'] = crval        # reference value for energy

In [15]:
hdulist = fits.open(f)
wcs_tab = WCS(refheader_tab)

ValueError: ERROR 5 in wcsset() at line 2170 of file cextern/wcslib/C/wcs.c:
Invalid parameter value.
ERROR 3 in tabset() at line 746 of file cextern/wcslib/C/tab.c:
Invalid tabular parameters: Each element of K must be positive, got 0.


In [19]:
np.diff(np.log(hdulist[1].data['ENERGY']))

array([ 0.31308868,  0.31308875,  0.31308871,  0.31308874,  0.31308873,
        0.31308875,  0.31308874,  0.31308874,  0.31308876,  0.31308871,
        0.31308871,  0.31308875,  0.31308872,  0.31308875,  0.3130887 ,
        0.31308875,  0.31308871,  0.31308868,  0.31308877,  0.31308875,
        0.31308873,  0.31308871,  0.31308877,  0.31308875,  0.31308869,
        0.3130887 ,  0.31308875,  0.31308873,  0.31308876])